## Assigment-01

### 基于模式匹配的对话机器人


### 1. Pattern Match
Pattern: (我想要A)     
Response:（如果你有A, 对你意味着什么呢？）     

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成 "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [10]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_match(pattern, saying):
    "Define a simple proceture that checks whether the saying matches with the pattern"
    
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] !=saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

# This isalpha() returns True if all characters in the strings are alphabets and 
# False if at least one character is not alphabet.

In [2]:
pat_match('I want ?X'.split(), 'I want holiday'.split())

True

In [3]:
pat_match('I dreamed about ?X'.split(), 'I have dreamed a dog'.split())

False

In [4]:
def pat_match(pattern, saying):
    "Revise the previous procedure to return the matched variable <A>."
    
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])


In [5]:
pat_match('I want ?X'.split(), 'I want holiday'.split())

('?X', 'holiday')

In [6]:
pat_match('?X equals ?X'.split(), '2+2 equals 4'.split())
# Note that when we have two pairs matched variables, the procedure only returns the first one

('?X', '2+2')

In [7]:
def pat_match(pattern, saying):
    "Revise the previous procedure so that we can identify all matched pairs."
    
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])
        

In [8]:
pat_match('?X is greater than ?Y'.split(), '3 is greater than 2'.split())

[('?X', '3'), ('?Y', '2')]

In [111]:
# Now we want to do two more things --- a) define a procedure that translates the matched relations
# into a dictionary; and b) define a procedure that can substitute variables given a dictionary
def pat_to_dic (patterns):
    return{k:v for k, v in patterns}

def substitute(pattern, pat_dic):
    if not pattern: return []
    
    return [pat_dic.get(pattern[0], pattern[0])] + substitute(pattern[1:], pat_dic)


In [10]:
got_patterns = pat_match('I want ?X'.split(), 'I want iPhone'.split())

In [11]:
got_patterns

[('?X', 'iPhone')]

In [13]:
substitute("What do you do if you got a ?X".split(), pat_to_dic(got_patterns))

['What', 'do', 'you', 'do', 'if', 'you', 'got', 'a', 'iPhone']

In [14]:
' '.join(substitute("What do you do if you got a ?X".split(), pat_to_dic(got_patterns)))
# join the sentence together

'What do you do if you got a iPhone'

In [15]:
# try another example
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [17]:
' '.join(substitute('Why does ?P need ?X'.split(), pat_to_dic(john_pat)))

'Why does John need vacation'

In [77]:
# Now we can define a pattern dictionary
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

[key.split() for key in defined_patterns.keys()]
defined_patterns.get("I need ?X")

['Image you will get ?X soon', 'Why do you need ?X ?']

In [91]:
import random
def get_response(saying, pat_dic):
    "Define a procedure that returns a patterned response based the dictionary supplied"
    
    if not saying or not pat_dic: return []
    
    response_list = []
    
    for key, value in pat_dic.items():
        got_patterns = pat_match(key.split(), saying.split())
        if got_patterns:
            pat_response = pat_dic.get(key)
            for r in pat_response:
                response_list.append(' '.join(substitute(r.split(), pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "Sorry, I don't know how to answer."


In [95]:
get_response('My mom told me something', defined_patterns)

'Talk about more about your mom'

In [96]:
get_response('I need vacation', defined_patterns)

'Why do you need vacation ?'

### 2. Segment Match
我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable.


In [9]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [26]:
is_pattern_segment('?*PYAMC')

True

In [43]:
from collections import defaultdict

fail = [True, None]


def pat_match_with_seg(pattern, saying):
    "Revise the previous pat_match in order to match ?*x variable with a segment of texts."
    
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):    # when we have a ?X variable
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):   # when we have a ?*X variable
        if segment_match(pattern, saying) != fail:
            match, index = segment_match(pattern, saying)
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else: 
            return segment_match(pattern, saying) # when the segment_match returns fail
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    

def segment_match(pattern, saying):
    "Find the group of words in saying that matches with the ?*x variable in a given pattern."
    
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')  # change the ?* prefix into a single ?
    
    if not rest: return (seg_pat, saying), len(saying) 
    
    for i, token in enumerate(saying):
        if rest[0] == token: 
            return (seg_pat, saying[:i]), i    # note i = len(saying) :)
    
    return fail # to prevent ?*X matches the whole of saying

In [44]:
segment_match('?*P is very good'.split(), "My dog along with my cat is very good".split())

(('?P', ['My', 'dog', 'along', 'with', 'my', 'cat']), 6)

In [45]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [46]:
pat_match_with_seg('?*P are good'.split(), "My cat is very cute".split())

[True, None]

In [47]:
# Give some example patterns in order to construct responses
response_pair = {
    'I need ?X': ['Why do you need ?X'],
    "I don't like my ?X": ["What bad things did ?X do for you?"]
}

In [48]:
pat_match_with_seg('I need ?*X'.split(), "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [88]:
got_pattern = pat_match_with_seg('I need ?*X'.split(), "I need an iPhone".split())
substitute("Why do you need ?X".split(), pat_to_dic(got_pattern))

['Why', 'do', 'you', 'need', 'an iPhone']

In [126]:
# Revise the pat_to_dic procedure to return a joined phrase for the substituted values of ?X
#  instead of broken words!

def pat_to_dic(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k,v in patterns} 
# join v if v is a list

In [89]:
substitute("Why do you need ?X".split(), pat_to_dic(got_pattern))
' '.join(substitute("Why do you need ?X".split(), pat_to_dic(got_pattern)))

'Why do you need an iPhone'

In [52]:
# If we have a pattern like this one:

("?*X hello ?*Y", "Hi, how do you do")
pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())

[('?X', ['I', 'am', 'mike,'])]

In [136]:
got_pattern = pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())
substitute("Hi, how do you do?".split(), pat_to_dic(got_pattern))

['Hi,', 'how', 'do', 'you', 'do?']

In [60]:
rules = {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    "?*X hello ?*Y": ["Hi, how do you do?"]
}

#### 问题 1
编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [127]:
# Now we want to define a procedure, get_response, that does the following things:
#  -- takes two inputs, a "saying", and a dictionary that specify a set of response rules，
#     where each element in the dictionary follows the format 
#     {<pattern>: [<response1>,<response2>,...]}
#  -- it finds a matched pattern within the dictionary supplied for the "saying" (pat_match_with_seg)
#  -- it finds the key words within the "saying" used as substitute and gives a patterned response (substitute, pat_to_dic)
#  -- to make the response interesting, it randomly chooses a patterned response for a list of possible responses defined by the rules


import random

def get_response(saying, pat_dic):
    if not saying or not pat_dic: return []
    
    response_list = []
    
    for key, value in pat_dic.items():
        got_patterns = pat_match_with_seg(key.split(), saying.split())
        if got_patterns != fail:
            pat_response = pat_dic[key]
            for r in pat_response:
                response_list.append(' '.join(substitute(r.split(), pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "Sorry, I don't know how to answer."


In [131]:
pat_match_with_seg("?*X hello ?*Y".split(), "I was a scientist".split())

[True, None]

In [138]:
segment_match("?*X hello ?*Y".split(), "I am mike, hello".split())

(('?X', ['I', 'am', 'mike,']), 3)

In [121]:
pat_match_with_seg("I was ?*X".split(), "I was a scientist".split())

[('?X', ['a', 'scientist'])]

In [128]:
get_response("I was a historian", rules)

'Were you really a historian ?'

In [129]:
get_response("Hi, my name is John", rules)

"Sorry, I don't know how to answer."

In [140]:
get_response("I am Mike, hello", rules)

'Hi, how do you do?'

#### 问题 2
将以上程序改写为能够支持中文输入的模式（提示：jieba分词）

In [241]:
import jieba
import random
from collections import defaultdict

fail = [True, None]

def is_variable(pat):  
    "Check if the input string is a pattern variable ?x"
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def is_pattern_segment(pattern):
    "Check if the input string is a segment pattern variable ?*x"
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def pat_to_dic(patterns):
    "Returns a dictionary that maps the pattern variables to key words identified in the sentence"
    return {k: ''.join(v) if isinstance(v, list) else v for k,v in patterns} 

def substitute(pattern, pat_dic):
    "Returns a response after replacing the pattern variables with the key words"
    if not pattern: return []
    return [pat_dic.get(pattern[0], pattern[0])] + substitute(pattern[1:], pat_dic)

def pat_match_with_seg(pattern, saying):
    "Match the pattern variables with a segment of texts and return the matched result"
    
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):    # when we have a ?X variable
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):   # when we have a ?*X variable
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])  
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    
def segment_match(pattern, saying):
    "Find the group of words in saying that matches with the ?*x variable in a given pattern."
    
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')  # change the ?* prefix into a single ?
    
    if not rest: return (seg_pat, saying), len(saying) 
    
    for i, token in enumerate(saying):
        if rest[0] == token: 
            return (seg_pat, saying[:i]), i # note i = len(saying)
    
    return fail # modified - to prevent ?*X matches the whole of saying


def get_response(saying, pat_dic):
    """
    Define a procedure that does the following things:
    - Match the saying with keys in the pattern dictionary supplied;
    - Fetch a response from the dictionary;
    - Replace the pattern variable with the key words in saying and return a response
    """
    if not saying or not pat_dic: return []
    
    response_list = []
    saying_token = ' '.join(jieba.cut(saying)).split() # split the saying into tokens
    
    for key, value in pat_dic.items():
        key_str = ' '.join(jieba.cut(key)) # output a string with whitespaces
        key_str_token = key_str.replace("? * x", "?*x").replace("? * y", "?*y").replace("? * z", "?*z").split()
        # remove the whitespaces and split the string into tokens
        got_patterns = pat_match_with_seg(key_str_token, saying_token)
        
        if not None in got_patterns:  # to prevent we get any 'fail' from got_patterns 
        # (note that this is a strict condition -- pat_match_with_seg may find multiple variables but only one of them fails)
            pat_response = pat_dic[key]
            for r in pat_response:
                r_token = ' '.join(jieba.cut(r)).replace("? x", "?x").replace("? y", "?y").replace("? z", "?z").split()
                response_list.append(''.join(substitute(r_token, pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "对不起，我没有明白你的意思，能稍微解释一下吗？"

In [242]:
rules_dic = {
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？']
}



In [243]:
get_response("你似乎总是这样", rules_dic)

'例如什么时候?'

In [244]:
get_response("我昨天看到机器人在博物馆里", rules_dic)

'你为什么觉得机器人要解决你的问题？'

In [246]:
get_response("葡萄和草莓一样好吃", rules_dic)

'好吃会对你有什么影响呢?'

In [248]:
get_response("其实我是外星人", rules_dic)

'其实想告诉你，或许我早就知道你是外星人'

In [250]:
get_response("如果生活欺骗了你", rules_dic)

'真的吗？如果生活欺骗了你的话'

In [251]:
get_response("忧愁就像一江春水向东流", rules_dic)

'忧愁和一江春水向东流真的有关系吗？'

In [228]:
# Test cases!

pat_str = ' '.join(jieba.cut("?*x总是?*y")) # output a string with whitespaces
pat_list = pat_str.replace("? * x", "?*x").replace("? * y", "?*y").replace("? * z", "?*z").split()
pat_list
is_pattern_segment(pat_list[0])

seg_list = ' '.join(jieba.cut("我昨天看到机器人在博物馆里")).split() # split the string into a list
segment_match(pat_list, seg_list)
pat_match_with_seg(pat_list, seg_list)


#fail in [('?x', []), True, None]
#for k,v in got_patterns:
#    print({k: ''.join(v) if isinstance(v,list) else v})
#substitute('你为什么要提机器人的事情?'.split(), pat_to_dic(pat_match_with_seg(pat_list, seg_list)))

True

In [252]:
# More test cases....

pat_str = ' '.join(jieba.cut("?*x和?*y一样?*z")) # output a string with whitespaces
pat_list = pat_str.replace("? * x", "?*x").replace("? * y", "?*y").replace("? * z", "?*z").split()
pat_list
seg_list = ' '.join(jieba.cut("葡萄和草莓一样好吃")).split() # split the string into a list
seg_list
segment_match(pat_list, seg_list)
patterns = pat_match_with_seg(pat_list, seg_list)
pat_to_dic(patterns)
' '.join(jieba.cut('?z会对你有什么影响呢?')).replace("? z", "?z").split()



['?z', '会', '对', '你', '有', '什么', '影响', '呢', '?']

In [ ]:
# Failed experiments for stripping the whitespaces between non-chinese characters after jieba.cut

#jieba.suggest_freq(('?*x'), True)
#jieba.load_userdict('user_dict.txt')

#import re
#re.split('(\?* )x', pat_list)

#for i, token in enumerate(pat_list):
#    if token.isalpha():
#        print (pat_list[ :i+1].replace(" ", "")) # replace whitespaces

#"机器".isalpha()

#### 问题 3 
多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

【待补 ：）】

#### 问题4

1. 这样的程序有什么优缺点？你有什么可以改进的方法吗？
   + **优点**： 基于规则驱动，搭建过程简单明了；功能有限，容易排错；可以随时添加和修改规则。
   + **缺点**： 规则过于僵硬化不够变通，比如用户在输入过程中多加一个空格或标点符号，可能就无法识别了；另外这种识别关键词的算法在pattern match的时候似乎不够有效率（也可能是我写的for-loop的解决方案不够有效率），每次在识别匹配的pattern的时候，需要搜索整个词典（如果词典数据库非常庞大，反应时间就会慢很多）。
   + **改进想法**：如果能够基于类似于课上例子语法树的grammar rule来建立规则，可能适用空间和变化范围会广泛一些（比如判断用户输入的是名词、动词、形容词、标点等等，从而组合做出应答）。如果用来存储 pattern:response 的词典能够基于某种规则进行优化（比如任务型分类），或许可以减少搜索时间从而增加应答速度。不清楚目前业界的客服机器人所用的主流的数据存储模式是什么？以及是如何优化机器人的应答速度和应变能力的呢？   
   
   

2. 什么是数据驱动？数据驱动在这个程序里如何体现？
   + 联想上课举的例子，通过修改基于规则的语法树来随机产生越来越复杂的语句和算数公式，数据驱动（data-driven）的思维方式是最终需要程序能够针对环境作出调整变化，对于类似但没有见过的情况作出正确的解析和回应。在这个程序里，数据驱动其实是由人为修改程序语句实现的 --- 先以简单的英文语句pattern作为起始点，写出一个对话交互的程序，然后拓展到兼容中文模式；程序的内核没有变（pattern identification ---> pattern match ---> retrieve & complie response）但程序的实现方式由于数据的不同而有改动。


3. 数据驱动与AI的关系是什么？
   + 一个最终获得人工智能的机器或程序，或者，一个“learning machine”（借用Turing文章里的概念），是能够从数据中习得一种 time invariant rule 来应对数据的变化（Turing给的类比例子是美国的宪法），比如，通过一种自然选择的过程（time invariant rule）来实现程序自身的变化 （time variant operation） ---- 那些运行速度快的程序有更大概率被重复使用，从而淘汰掉运行慢的程序。数据驱动的模式应该是机器习得“智能”的一个必需但非充分条件 --- Turing自己也提到了离散状态的机器（discrete-state machines）所能具有的“状态”是有限度的，无论其是否受存储能力的限制，因为这些“状态”组成的空间是可预测的，而真正的“智能”在于如何将一种随机的、甚至“不可预测（unpredictable）” 的状态赋予机器（类似于自然进化中的基因变异），而这种不可预测的状态恰好给出了机器自身进化中的最优解（基因变异的某些物种反而成为适者生存下来）。